In [1]:
# this code creates the spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
import numpy as np

 Map-Reduce: Gradient descent





Create a function `f_linear(b, x)` that receives the parameters `b` and one data point `x` as lists and return the prediction for that data point. Assume that the first element of `b` is the intercept.

In [2]:
# define below the function `f_linear` which performs a linear prediction based on parameters as data point
def f_linear(b, x):
    y = b[0]
    for i in range(len(x)):
        y = y + b[i+1]*x[i]
    return(y)
    #raise NotImplementedError()

In [3]:
# for the example above, if we assume b = [0, 1], and the first data point x = [1], y = 2
f_linear([0, 1], [2])

2


Define the function `L(y_pred, y)` that receives a prediction `y_pred` and the actual value `y` and returns the squared error between them.

In [5]:
def L(y_pred, y):
    z = (y - y_pred)**2
    return(z)
    #raise NotImplementedError()

In [6]:
# there should be not error here
L(1, 1)

0


Create a function `gf_linear(f, b, x, y)` which returns the gradient of the linear function `f` with parameter `b` with respect to the squared loss function, evaluated at `x` and the actual outcome `y`. This function should return a vector with each element $j$ corresponding to the gradient with respect $b_j$, with $j = \{0, 1, \dots, p\}$.

In [9]:
# define `gf_linear`
def gf_linear(f, b, x, y):
    f = f_linear(b,x)
    gd = []
    db_0 = 2*(f-y)
    gd.append(db_0)
    for i in range(len(x)):
        dbi = 2*x[i]*(f-y)
        gd.append(dbi)
    return(gd)
    

In [10]:
# for the example above and first data point
x = [1]
y = 2
b = [0, 1]
gf_linear(f_linear, b, x, y)

[-2, -2]

In [11]:
# for the example above and second data point
x = [-2]
y = -1
b = [0, 1]
f=f_linear(b,x)
gf_linear(f, b, x, y)

[-2, 4]



Develop a map-reduce job that produces a value so that the first element of the value is the mean loss function across all the data. 

In [13]:
# data
rdd_data = sc.parallelize([
    [[1, 2], 3],
    [[3, 1], 4],
    [[-1, 1.5], 0],
    [[-9, 3], 0]
])

In [14]:
# create function `map_mse` below
def map_mse(f, b, L, xy):
    f=f_linear(b,xy[0])
    L=L(f,xy[1])
    return([0,[L,1]])
    #raise NotImplementedError()

You should apply the map function in the following way:

```python
# for an example set of `b = [0, 0, 0]`
rdd_data.map(lambda x: map_generator(f_linear, [0, 0, 0], L, x))
```


In [15]:

rdd_data.map(lambda x: map_mse(f_linear, [0, 0, 0], L, x)).collect()

[[0, [9, 1]], [0, [16, 1]], [0, [0.0, 1]], [0, [0, 1]]]

You will now create a reduce job that receives two values of a previous reduce (or map) and merge them appropriately. Remember that at the end of the reduce job, the first element of the value should be the mean squared error. Create the function `reduce_mse(v1, v2)` below.

In [17]:
# create function `reduce_mse` below
def reduce_mse(v1, v2):
    l1,l2 = v1
    t1,t2 = v2
    rd = [(l1*l2+t1*t2)/(l2+t2),l2+t2]
    return(rd)
   

In [18]:
# the following function call should return the mean squared error
rdd_data.\
    map(lambda x: map_mse(f_linear, [0, 0, 0], L, x)).\
    reduceByKey(reduce_mse).first()[1][0]

6.25

In [19]:
# the following function call should return the mean squared error
rdd_data.\
    map(lambda x: map_mse(f_linear, [2, 2, 3], L, x)).\
    reduceByKey(reduce_mse).first()[1][0]

41.8125

you will compute the cumulative gradient of a model on the data. You will define a map function `map_gradient(f, gf, b, xy)` that would receive a function `f`, its gradient `gf`, its parameters `b`, and a data point `xy = [x, y]`. Also you will define a function `reduce_gradient(v1, v2)` that combines the two values appropriately. In the map function, you probably do not need to keep extra values beyond the actual gradient.

In [21]:
# define the function `map_gradient` below
def map_gradient(f, gf, b, xy):
    gf = gf_linear(f,b,xy[0],xy[1])
    return([0,gf])
    #raise NotImplementedError()

In [23]:
# define the function `reduce_gradient` below
def reduce_gradient(v1, v2):
    rg=[]
    for i in range(len(v1)):
        rg.append(v1[i]+v2[i])
    return(rg)
    #raise NotImplementedError()

In [24]:
rdd_data.map(lambda xy: map_gradient(f_linear, gf_linear, [0, 0, 0], xy)).\
    reduceByKey(reduce_gradient).first()[1]

[-14.0, -30.0, -20.0]

 we can run an optimization below, and the MSE should decrease with each iteration

In [26]:
b = [0, 0, 0]
learning_rate = 0.05
print("Initial solution: \t", b)
for _ in range(10):
    print("New iteration")
    print("=============")
    gradient = rdd_data.map(lambda xy: map_gradient(f_linear, gf_linear, b, xy)).\
        reduceByKey(reduce_gradient).first()[1]
    b = [b0 - learning_rate*g0 for b0, g0 in zip(b, gradient)]
    print("Current solution: \t", b)
    mse = rdd_data.\
        map(lambda x: map_mse(f_linear, b, L, x)).\
        reduceByKey(reduce_mse).first()[1][0]
    print("Current MSE: \t\t", mse)
    
    

Initial solution: 	 [0, 0, 0]
New iteration
Current solution: 	 [0.7000000000000001, 1.5, 1.0]
Current MSE: 		 25.702500000000004
New iteration
Current solution: 	 [1.2700000000000002, -8.03, 3.375]
Current MSE: 		 1941.2104390624995
New iteration
Current solution: 	 [-5.887249999999999, 76.03925, -20.932375]
Current MSE: 		 154868.55739933692
New iteration
Current solution: 	 [58.490481249999995, -674.7452812500001, 197.19040937500003]
Current MSE: 		 12361757.656351853
New iteration
Current solution: 	 [-516.9456870312499, 6032.90305703125, -1751.7632777343752]
Current MSE: 		 986733680.53671
New iteration
Current solution: 	 [4624.096880300781, -53895.11618255079, 15660.883497880859]
Current MSE: 		 78762540125.70647
New iteration
Current solution: 	 [-41307.57420476065, 481518.98704511696, -139908.6478753955]
Current MSE: 		 6286942318447.336
New iteration
Current solution: 	 [369059.0336107604, -4302024.060799995, 1249994.2051317175]
Current MSE: 		 501833024339880.2
New iteration

In [27]:
b = [0, 0, 0]
learning_rate = 0.05
print("Initial solution: \t", b)
for _ in range(10):
    print("New iteration")
    print("=============")
    gradient = rdd_data.map(lambda xy: map_gradient(f_linear, gf_linear, b, xy)).\
        reduceByKey(reduce_gradient).first()[1]
    b = [b0 - learning_rate*g0 for b0, g0 in zip(b, gradient)]
    print("Current solution: \t", b)
    mse = rdd_data.\
        map(lambda x: map_mse(f_linear, b, L, x)).\
        reduceByKey(reduce_mse).first()[1][0]
    print("Current MSE: \t\t", mse)
#raise NotImplementedError()

Initial solution: 	 [0, 0, 0]
New iteration
Current solution: 	 [0.7000000000000001, 1.5, 1.0]
Current MSE: 		 25.702500000000004
New iteration
Current solution: 	 [1.2700000000000002, -8.03, 3.375]
Current MSE: 		 1941.2104390624995
New iteration
Current solution: 	 [-5.887249999999999, 76.03925, -20.932375]
Current MSE: 		 154868.55739933692
New iteration
Current solution: 	 [58.490481249999995, -674.7452812500001, 197.19040937500003]
Current MSE: 		 12361757.656351853
New iteration
Current solution: 	 [-516.9456870312499, 6032.90305703125, -1751.7632777343752]
Current MSE: 		 986733680.53671
New iteration
Current solution: 	 [4624.096880300781, -53895.11618255079, 15660.883497880859]
Current MSE: 		 78762540125.70647
New iteration
Current solution: 	 [-41307.57420476065, 481518.98704511696, -139908.6478753955]
Current MSE: 		 6286942318447.336
New iteration
Current solution: 	 [369059.0336107604, -4302024.060799995, 1249994.2051317175]
Current MSE: 		 501833024339880.2
New iteration

Learning rate determines how slowly or fast we will move towards the optimal solution. If it is too small we may need many iterations to find optimal value. If it is too large, wemay skip optimal value. For value of 0.05, the value increases till a point and then decreases.